In [ ]:
### 使用提示词完成任务

In [1]:
# 导入依赖库
import dashscope
import os

# 从环境变量中获取 API Key
dashscope.api_key = os.getenv('DASHSCOPE_API_KEY')

# 基于 prompt 生成文本
# 使用 deepseek-v3 模型
def get_completion(prompt, model="deepseek-v3"):
    messages = [{"role": "user", "content": prompt}]    # 将 prompt 作为用户输入
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format='message',  # 将输出设置为message形式
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content  # 返回模型生成的文本
    
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
# 目标
{instruction}

# 用户输入
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# 调用大模型
response = get_completion(prompt)
print(response)

==== Prompt ====

# 目标

你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。


# 用户输入

办个100G的套餐。


# 分析结果

根据用户输入"办个100G的套餐"，可以识别出以下选择条件：

1. 月流量需求：用户明确要求100GB的月流量
   - 识别属性：月流量
   - 识别值：100GB

2. 其他属性：
   - 名称：用户未提及具体套餐名称要求
   - 月费价格：用户未提及价格范围或限制

因此，用户的主要选择标准是月流量达到100GB，对其他属性没有明确限制。


### JSON格式

In [2]:
# 输出格式
output_format = """
以 JSON 格式输出
"""

# 稍微调整下咒语，加入输出格式
prompt = f"""
# 目标
{instruction}

# 输出格式
{output_format}

# 用户输入
{input_text}
"""

# 调用大模型，指定用 JSON mode 输出
response = get_completion(prompt)
print(response)

```json
{
  "名称": null,
  "月费价格": null,
  "月流量": "100G"
}
```


### CoT示例

In [4]:
instruction = """
给定一段用户与手机流量套餐客服的对，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

#cot = ""
cot = "请一步一步分析对话"

prompt = f"""
# 目标
{instruction}
{cot}

# 输出格式
{output_format}

# 对话上下文
{context}
"""

response = get_completion(prompt)
print(response)

# 逐步分析对话

1. **礼貌性检查**：
   - 客服使用了"亲"作为称呼，这属于网络用语，不符合官方口吻要求
   - 虽然态度友好，但不符合"必须用官方口吻"的规范

2. **官方口吻检查**：
   - 使用了"亲"这样的网络用语
   - 不符合"不能使用网络用语"的要求

3. **信息准确性检查**：
   - 提到了产品名称"无限套餐"
   - 准确说明了月费300元
   - 准确说明了月流量1000G
   - 信息完整准确

4. **话题延续性检查**：
   - 以"您感兴趣吗？"结尾，是开放式问题
   - 不是话题终结者

# 不符合规范的主要问题
- 使用了网络用语"亲"，违反了官方口吻的要求

# 最终判断
N


### 使用Prompt调优Prompt

In [5]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
# 目标
{instruction}

# 用户提示词
{user_prompt}
"""

response = get_completion(prompt)
print(response)

# 优化后的提示词

作为手机流量套餐客服代表"小瓜"，你需要：
1. 热情友好地介绍自己，明确服务范围
2. 主动询问用户需求(如每月流量使用量、预算范围、是否在校生等)
3. 根据用户需求推荐最合适的套餐，完整说明套餐内容(月费、流量、特殊限制)
4. 比较不同套餐的性价比，突出推荐理由
5. 提供办理方式和后续服务信息
6. 保持专业客服语气，使用亲切但不随意的语言风格

可推荐套餐：
- 经济套餐：50元/月，10G流量
- 畅游套餐：180元/月，100G流量 
- 无限套餐：300元/月，1000G流量
- 校园套餐：150元/月，200G流量(仅限在校生)

# 优化建议

原提示词提供了基本框架但缺乏细节指导。优化后：
1. 明确了交互流程和客服行为规范
2. 增加了用户需求分析环节，使推荐更精准
3. 强调了比较说明的重要性，帮助用户做决策
4. 细化了客服语气和风格要求
5. 补充了服务闭环(办理方式和后续服务)
6. 结构化呈现套餐信息便于快速参考

建议进一步优化方向：
- 可添加常见问题应对策略(如流量不够用怎么办)
- 考虑不同用户类型(学生/商务人士/老年人)的沟通方式差异
- 增加促销活动等动态信息处理能力

# 问题示例

1. 您平均每月使用多少流量？这样我可以为您推荐最合适的套餐
2. 您的预算范围大概是多少？我们有不同价位的套餐可供选择
3. 请问您是在校学生吗？我们有针对学生的专属优惠套餐
